In [35]:
import pandas as pd
import os
import copy
import unittest
import string
import re

def read_file(trainFile):
    pwd = os.getcwd()
    os.chdir(os.path.dirname(trainFile))
    file1 = open(trainFile, 'r') 
    lines = file1.read().splitlines()
    return lines

In [36]:
lines = read_file("C:/Code/advent/2021_day21_input.txt")

In [37]:
lines_example1 = read_file("C:/Code/advent/2021_day21_example1.txt")
lines_example1

['Player 1 starting position: 4', 'Player 2 starting position: 8']

In [38]:
def parse(input):
    result = []
    for line in input:
        value = int(line.split(': ')[1])
        result.append(value)
    return result

parse(lines_example1)

[4, 8]

In [39]:
def calculate_next_simple(start, roll):
    return (start + roll - 1) % 10 + 1

def calculate_next(start,outcomes):
    return calculate_next_simple(start, sum(outcomes))

def advance(current, player, outcomes):
    result = [current[0],current[1]]    
    result[player] = calculate_next(current[player],outcomes)
    return result

def roll(first):
    result = []
    for i in range(3):
        result.append((first + i- 1)%100 + 1)
    return result    

def calculate_part_one(input):
    current = [input[0],input[1]]
    next_roll = 1
    last_roll = 1
    finished = False
    active = 0
    total_score = [0,0]
    while not finished:
        outcomes = roll(next_roll)
        last_roll = next_roll + 2
        next_roll += 3
        current = advance(current, active, outcomes)
        total_score[active] += current[active]
        #print(active+1, outcomes, current[active], total_score[active])
        if max(total_score) >= 1000:
            finished = True
        active = 1-active

    return min(total_score) * last_roll
    
calculate_part_one(parse(lines_example1))

739785

In [40]:
calculate_part_one(parse(lines))

913560

In [41]:
def generate_dirac_outcomes(depth):
    if depth > 1:
        start = generate_dirac_outcomes(depth-1)
    else:
        start = {}
        start[0] = 1
    result = {}
    for i in range(1,4):
        for v in start:
            result[i+v] = result.get(i+v,0) + start[v]
    return result

print(generate_dirac_outcomes(1))
print(generate_dirac_outcomes(2))
print(generate_dirac_outcomes(3))
dirac_outcomes = generate_dirac_outcomes(3)

{1: 1, 2: 1, 3: 1}
{2: 1, 3: 2, 4: 3, 5: 2, 6: 1}
{3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}


In [45]:
def find_lowest_key(input, hint):
    lowest = 1000
    lowest_key = None
    for i in input:
        score = i[2]
        value = min(score)
        if value < lowest:
            lowest_key = i
            lowest = value
        if value == hint:
            return lowest_key
    return lowest_key

def generate_next_state_with_outcome(state, roll):    
    (position, active, score) = state
    new_active = 1 - active
    result = None
    new_position = calculate_next_simple(position[active], roll)
    if active == 0:
        result = ((new_position, position[1]),new_active,(score[0]+new_position,score[1]))
    else:
        result = ((position[0],new_position),new_active,(score[0],score[1]+new_position))

    #print('generate', state, roll , '---->', result)
    return result

def generate_next(state, multiplier):
    result = {}
    for outcome in dirac_outcomes:
        outcome_count = dirac_outcomes[outcome]
        key = generate_next_state_with_outcome(state, outcome)
        result[key] = outcome_count * multiplier
    return result

def calculate_part_two(input):
    current = ((input[0],input[1]), 0, (0,0))
    unexplored = {}
    unexplored[current] = 1
    wins = [0,0]
    max_iterations = 200000
    n = 0
    lowest_print = 0
    while len(unexplored) > 0 and max_iterations > n:
        n += 1
        #print('step', n, unexplored)
        lowest_key = find_lowest_key(unexplored, lowest_print)
        lowest_value = unexplored[lowest_key]
        if lowest_print < min(lowest_key[2]):            
            lowest_print = min(lowest_key[2])
            print(lowest_print, n, wins, lowest_key)
        unexplored.pop(lowest_key)
        copies = generate_next(lowest_key, lowest_value)
        for c in copies:
            count = copies[c]
            score = c[2]
            if score[0] >= 21:
                wins[0] = wins[0] + count
            elif score[1] >= 21:
                wins[1] = wins[1] + count
            else:
                unexplored[c] = unexplored.get(c,0) + count
        #print(unexplored)

    print(unexplored)
    print(len(unexplored), n, wins)
    return None

In [46]:
calculate_part_two(parse(lines_example1))

1 9 [0, 0] ((7, 1), 0, (7, 1))
2 71 [0, 0] ((7, 2), 0, (7, 2))
3 131 [0, 0] ((7, 3), 0, (7, 3))
4 401 [13824, 0] ((7, 4), 0, (7, 4))
5 662 [96768, 0] ((7, 5), 0, (7, 5))
6 1279 [322560, 0] ((7, 6), 0, (7, 6))
7 2074 [3621192, 64010] ((7, 7), 0, (7, 7))
8 3762 [15921768, 318320] ((10, 7), 0, (17, 8))
9 5972 [123724286, 9250826] ((10, 8), 0, (17, 9))
10 8362 [855955300, 59996552] ((10, 9), 0, (17, 10))
11 11149 [4324364844, 274487493] ((10, 10), 0, (17, 11))
12 14047 [19970131934, 8272987017] ((10, 10), 0, (17, 12))
13 16868 [90675799793, 77832313849] ((10, 10), 0, (17, 13))
14 19580 [426273358701, 459609726791] ((10, 10), 0, (17, 14))
15 22001 [1818828593029, 1950263772764] ((10, 10), 0, (17, 15))
16 24113 [6737120603807, 6622801365133] ((10, 10), 0, (17, 16))
17 25896 [21401244356923, 18967663817339] ((10, 10), 0, (17, 17))
18 27289 [59288284745124, 48540600335876] ((3, 10), 0, (20, 18))
19 28289 [144280143657819, 113356675653499] ((8, 10), 0, (19, 19))
20 28889 [300506516528403, 23238

In [44]:
calculate_part_two(parse(lines))

1 9 [0, 0] ((1, 4), 0, (1, 4))
2 16 [0, 0] ((2, 4), 0, (2, 4))
3 23 [0, 0] ((3, 4), 0, (3, 4))
4 86 [0, 0] ((7, 4), 0, (7, 4))
5 193 [0, 0] ((7, 5), 0, (7, 5))
6 508 [27648, 0] ((7, 6), 0, (7, 6))
7 1061 [138240, 146039] ((7, 7), 0, (7, 7))
8 2165 [1140306, 726248] ((8, 8), 0, (8, 8))
9 3568 [6712464, 11574485] ((9, 9), 0, (9, 9))
10 5267 [26078552, 70163198] ((10, 10), 0, (10, 10))
11 7397 [199491102, 292541795] ((8, 7), 1, (11, 11))
12 9814 [2089675810, 2407600424] ((9, 8), 1, (12, 12))
13 12374 [16874108605, 17818903873] ((10, 9), 1, (13, 13))
14 14942 [111068414390, 104726270807] ((10, 10), 0, (17, 14))
15 17356 [515285652886, 469921641328] ((10, 10), 0, (17, 15))
16 19502 [1805195217051, 1718786739595] ((10, 10), 0, (17, 16))
17 21298 [5190041187595, 5321046884108] ((10, 10), 0, (17, 17))
18 22697 [13334784580547, 14502276074914] ((10, 8), 1, (18, 18))
19 23697 [31296949047932, 35344663629407] ((10, 9), 1, (19, 19))
20 24297 [64481156330016, 74361222990436] ((10, 7), 1, (20, 20))
